In [2]:
import os
import datetime as dt
import numpy as np
import pandas as pd
import alpaca_trade_api as tradeapi
from dotenv import load_dotenv
import quandl

In [3]:
#Pull 10Yr treasury data from Quandl
quandl_key = os.getenv("QUANDL_API_KEY")
mydata=quandl.get("USTREASURY/YIELD", authtoken=quandl_key, start_date="1900-12-31", end_date="2022-12-31")
mydata.head()


ten_years = mydata[['10 YR']]
ten_years.tail()


,10 YR
Date,
2021-03-11,1.54
2021-03-12,1.64
2021-03-15,1.62
2021-03-16,1.62
2021-03-17,1.63


In [4]:
load_dotenv()
# Set Alpaca API key and secret
alpaca_key = os.getenv('ALPACA_API_KEY')
alpaca_secret = os.getenv('ALPACA_SECRET_KEY')

api = tradeapi.REST(
    alpaca_key,
    alpaca_secret,
    api_version = "v2"
)

In [5]:
#Pull all data for each sector from alpaca - requires multiple pulls
#Concat data into one df
tickers = ["XLY", "XLP", "XLE","XLF", "XLV", "XLI", "XLB", "XLK", "XLU", "SPY"]

timeframe = "1D"
year_start=1990
year_list=[]
while year_start < 2022:
    year_list.append(year_start)
    year_start +=1

ticker_data = []
for my_year in year_list:
    
    start_date=str(my_year)+'-01-01'
    end_date=str(my_year)+'-12-31'

    data = api.get_barset(
    tickers,
    timeframe,
    start = pd.Timestamp(start_date, tz="America/New_York").isoformat(),
    end = pd.Timestamp(end_date, tz="America/New_York").isoformat(),
    limit = 1000
    ).df
    ticker_data.append(data)
    
all_ticker_data = pd.concat(ticker_data, axis="rows", join="inner")

all_ticker_data.index.rename("Date", inplace=True)
all_ticker_data.index = pd.to_datetime(all_ticker_data.index.date)


all_ticker_data.head()


SPY                                       XLB                \
              open    high     low   close     volume   open   high    low   
2008-01-02  146.53  146.99  143.88  144.55  175801806  41.70  41.95  40.98   
2008-01-03  144.92  145.49  144.12  144.46  111357758  41.52  42.17  41.45   
2008-01-04  143.34  143.44  140.92  140.93  195945264  41.42  41.57  40.70   
2008-01-07  141.81  142.23  140.10  141.46  220398905  40.81  41.06  39.67   
2008-01-08  142.10  142.90  138.63  138.81  303249870  40.40  41.07  39.55   

                             ...    XLV                                  XLY  \
            close    volume  ...   open   high    low  close   volume   open   
2008-01-02  41.33   9483635  ...  35.63  35.63  34.86  34.94  3915069  32.99   
2008-01-03  41.80   8699808  ...  34.98  35.30  34.56  35.18  2504282  32.11   
2008-01-04  40.74  13370213  ...  35.01  35.16  34.50  34.83  3214308  31.43   
2008-01-07  40.25  16971407  ...  34.82  35.58  34.82  35.57  5451740  31.14   
2008-01-08  39.60  21112177  ...  35.66  36.39  35.57  35.81  6210752  31.03   

                                            
             high    low  close     volume  
2008-01-02  32.99  32.11  32.13  3711800.0  
2008-01-03  32.22  31.68  31.78  9585427.0  
2008-01-04  31.58  30.75  30.78  5960834.0  
2008-01-07  31.18  30.68  30.93  6104205.0  
2008-01-08  31.31  30.31  30.32  4287009.0  

[5 rows x 50 columns]

In [6]:
#Slice only close prices and join with 10yr treasury pulled from qunadl
closing_prices = pd.DataFrame()
for ticker in tickers:
    closing_prices[ticker]=all_ticker_data[ticker]["close"]

    
combine_dataframe = pd.concat([closing_prices ,ten_years], axis="columns", join="inner")
combine_dataframe.index.name = 'Date'


combine_dataframe.to_csv("historical_data.csv")

combine_dataframe.head()

,XLY,XLP,XLE,XLF,XLV,XLI,XLB,XLK,XLU,SPY,10 YR
Date,,,,,,,,,,,
2008-01-02,32.13,28.34,79.75,28.41,34.94,38.37,41.33,26.10,41.98,144.55,3.91
2008-01-03,31.78,28.22,80.21,28.20,35.18,38.58,41.80,26.08,42.08,144.46,3.91
2008-01-04,30.78,28.04,78.01,27.39,34.83,37.60,40.74,25.07,42.30,140.93,3.88
2008-01-07,30.93,28.55,77.04,27.55,35.57,37.34,40.25,24.97,43.25,141.46,3.86
2008-01-08,30.32,28.38,76.08,26.54,35.81,36.39,39.60,24.21,43.33,138.81,3.86
